<a href="https://colab.research.google.com/github/alexsak/opencv_dl/blob/main/Mobile_document_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Step 1: Import required packages, set argument line.

Lines 2-7 handle importing the necessary Python packages that we’ll need.

Import four_point_transform function.

Import imutils module, containing functions for resizing, rotating, and cropping images. 

Import 'threshold_local' function from scikit-image, used to obtain the “black and white” feel to our scanned image.

Import NumPy for numerical processing, argparse  for parsing command line arguments, and cv2  for our OpenCV bindings.

Line 10-13: handle parsing our command line arguments. We’ll need only a single switch image, --image , which is the path to the image that contains the document we want to scan.

Step 2: Edge detection

Line 17: Load image with cv2.imread, using the input argument passed for "image"

Line 17-20: Resize image using mutils

Line 24: Convert image to grayscale
Line 25: Apply Gaussian blur 
Line 26: Perform Canny edge detection

Line 30-31: Display both images

Line 37: Find contours of edged image 
Line 38: Handled differened returned contours due to OpenCV version
Line 39: Sort contours by area and discard rest
Line 42: Loop through all contours
Line 44-45: Approximate number of points
Line 49: If contour has 4 points, assume we found the sought result
Line 55-56: Display the resulting contour

Line 62: Perform warping transformation, multiplying by resized ratio

Line 66-68: Take warped image, convert to grayscale and apply thresholding to get black and white feel 

Line 72-74: Display output

In [1]:
# import the necessary packages
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True,
	help = "Path to the image to be scanned")
args = vars(ap.parse_args())

# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread(args["image"])
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)

# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break

# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

ModuleNotFoundError: ignored